<a href="https://colab.research.google.com/github/arielmagbanua/image-classifiers/blob/main/mask-on-mask-off/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import os
import zipfile

# Extract and Prepare Dataset

1.) Mount your Google Drive

2.) Designate a directory where you want to extract your dataset (E.g. `/content/mask-on-mask-off`).


In [ ]:
# The path where to extract the dataset
COLAB_DATA_SET_PATH = '/content/mask-on-mask-off'

# clean up directory
!rm -R $COLAB_DATA_SET_PATH

data_zip = '/content/drive/MyDrive/mlds/dataset/mask_on_off.zip'
zip_ref = zipfile.ZipFile(data_zip, 'r')
zip_ref.extractall(COLAB_DATA_SET_PATH)
zip_ref.close()

# Constants / Parameters

In [ ]:
DESIRED_ACCURACY = 0.999

dataset_path = os.path.join(COLAB_DATA_SET_PATH)
print(dataset_path)

IMAGE_WIDTH = 320
IMAGE_HEIGHT = 320

/content/mask-on-mask-off


# Callbacks

In [ ]:
class Callbacks(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if logs.get('accuracy') > DESIRED_ACCURACY:
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True

# create the callbacks
callbacks = Callbacks()

# Build the Model

In [ ]:
model = tf.keras.models.Sequential([
  # input layer
  # first convolution
  tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3)),
  tf.keras.layers.MaxPooling2D(2, 2),

  # second convolution
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),

  # third convolution
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),

  # fourth convolution
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),

  # fifth convolution
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),

  # Flatten the results to feed into a DNN
  tf.keras.layers.Flatten(),

  # 512 neuron hidden layer
  tf.keras.layers.Dense(512, activation='relu'),

  # Only 1 output neuron.
  # It will contain a value from 0-1 where 0 for 1 class ('mask on') and 1 for the other ('mask off')
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_143 (Conv2D)          (None, 318, 318, 16)      448       
_________________________________________________________________
max_pooling2d_143 (MaxPoolin (None, 159, 159, 16)      0         
_________________________________________________________________
conv2d_144 (Conv2D)          (None, 157, 157, 32)      4640      
_________________________________________________________________
max_pooling2d_144 (MaxPoolin (None, 78, 78, 32)        0         
_________________________________________________________________
conv2d_145 (Conv2D)          (None, 76, 76, 64)        18496     
_________________________________________________________________
max_pooling2d_145 (MaxPoolin (None, 38, 38, 64)        0         
_________________________________________________________________
conv2d_146 (Conv2D)          (None, 36, 36, 64)      

# Compile the Model

In [ ]:
model.compile(
  loss='binary_crossentropy',
  optimizer=Adam(),
  metrics=['accuracy']
)

# Prepare Dataset with Data Generator

In [ ]:
# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(
  rescale=1./255,
  shear_range=0.2,
  zoom_range=0.2,
  height_shift_range=0.2,
  rotation_range=35,
  brightness_range=[0.2, 1.0],
  horizontal_flip=True,
  validation_split=0.2
)

# training data generator
train_generator = train_datagen.flow_from_directory(
  dataset_path,  # This is the source directory for training images
  target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),  # All images will be resized to 360x360
  batch_size=128,
  class_mode='binary',
  seed=7,
  subset='training'
)

# validation data generator
validation_generator = train_datagen.flow_from_directory(
  dataset_path,
  target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),  # All images will be resized to 360x360
  batch_size=32,
  class_mode='binary',
  seed=7,
  subset='validation'
)

Found 2870 images belonging to 2 classes.
Found 717 images belonging to 2 classes.


# Train / Fit

In [ ]:
history = model.fit(
  train_generator,
  validation_data=validation_generator,
  steps_per_epoch=8,
  epochs=100,
  verbose=1,
  # callbacks=[callbacks]
)

Epoch 1/100
8/8 [==============================] - 54s 7s/step - loss: 0.6965 - accuracy: 0.5600 - val_loss: 0.6739 - val_accuracy: 0.5007
Epoch 2/100
8/8 [==============================] - 55s 7s/step - loss: 0.6427 - accuracy: 0.6209 - val_loss: 0.4272 - val_accuracy: 0.8494
Epoch 3/100
8/8 [==============================] - 55s 7s/step - loss: 0.4727 - accuracy: 0.7801 - val_loss: 0.2821 - val_accuracy: 0.8940
Epoch 4/100
8/8 [==============================] - 52s 7s/step - loss: 0.2631 - accuracy: 0.8951 - val_loss: 0.4935 - val_accuracy: 0.8061
Epoch 5/100
8/8 [==============================] - 52s 7s/step - loss: 0.2300 - accuracy: 0.9206 - val_loss: 0.1944 - val_accuracy: 0.9344
Epoch 6/100
8/8 [==============================] - 54s 7s/step - loss: 0.1637 - accuracy: 0.9460 - val_loss: 0.1560 - val_accuracy: 0.9372
Epoch 7/100
8/8 [==============================] - 52s 7s/step - loss: 0.1248 - accuracy: 0.9556 - val_loss: 0.6728 - val_accuracy: 0.7252
Epoch 8/100
8/8 [==========

# Plot the Training History

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

#Evaluate the Model

In [ ]:
import pathlib
import PIL
import numpy as np

COLAB_EVAL_DATASET_PATH = '/content/drive/MyDrive/mlds/dataset/evaluation_data_set'

eval_ds = tf.keras.preprocessing.image_dataset_from_directory(
  COLAB_EVAL_DATASET_PATH,
  seed=123,
  image_size=(IMAGE_WIDTH, IMAGE_HEIGHT)
)

class_names = eval_ds.class_names
# print(class_names)

# plt.figure(figsize=(10, 10))

# for images, labels in eval_ds.take(1):
#   print('yo')
#   for i in range(10):
#     ax = plt.subplot(4, 4, i + 1)

#     image = images[i].numpy().astype("uint8")
#     plt.imshow(image)
#     plt.title(class_names[labels[i]])
#     plt.axis("off")

def infer(class_names, model, x):
  
  prediction = model.predict(x)
  result_index = np.argmax(prediction)
  probability = prediction[0][result_index]
  prediction_class_index = round(probability)

  return probability, class_names[prediction_class_index]

data_dir = pathlib.Path(COLAB_EVAL_DATASET_PATH)

plt.figure(figsize=(10, 10))

# mask on inference example
image_paths = list(data_dir.glob('mask_on/*.jpg'))
image = tf.keras.preprocessing.image.load_img(image_paths[0])
image = image.resize((IMAGE_WIDTH, IMAGE_HEIGHT))
image = tf.keras.preprocessing.image.img_to_array(image)
plt.subplot(2, 2, 1)
plt.imshow(image.astype("uint8"))

image_input = tf.expand_dims(image, axis=0)
pci, class_name = infer(class_names, model, image_input)
plt.title('{} = {}'.format(pci, class_name))


# mask off inference example
image_paths = list(data_dir.glob('mask_off/*.jpg'))
image = tf.keras.preprocessing.image.load_img(image_paths[4])
image = image.resize((IMAGE_WIDTH, IMAGE_HEIGHT))
image = tf.keras.preprocessing.image.img_to_array(image)

plt.subplot(2, 2, 2)
plt.imshow(image.astype("uint8"))

image_input = tf.expand_dims(image, axis=0)
pci, class_name = infer(class_names, model, image_input)
plt.title('{} = {}'.format(pci, class_name))





# Save the Model

In [ ]:
models_dir_path = COLAB_DATA_SET_PATH + '/models'
model_path = models_dir_path + '/moo.h5'

model.save(model_path)

# Convert the Model to Javascript

In [ ]:
!pip install tensorflowjs
from google.colab import files

!tensorflowjs_converter --input_format keras $model_path $models_dir_path

zip_path = COLAB_DATA_SET_PATH + '/moo_models.zip'

!zip -r $COLAB_DATA_SET_PATH $models_dir_path